In [141]:
from __future__ import division, generators, absolute_import, print_function

import numpy as np
import scipy.stats
import pandas as pd
from openpyxl import load_workbook
import random
data_dir = '../data/'

# read data colony

In [324]:
# RTS colony data
# code from Aaron

DATA_DIR = '../data/'
wb = load_workbook(DATA_DIR + 'Bishayee Colony Counts 10.27.97-3.8.01.xlsx') 

ws = wb.get_sheet_by_name('Sheet1')
rts_colony = []

for i in range(4, 1366):
    for col in ['D','E','F']:
        try:
            rts_colony.append(float(ws[col + str(i)].value))
        except:
            pass

In [328]:
# other invectogators colony data
# code from Aaron


DATA_DIR = '../data/'
wb = load_workbook(DATA_DIR + 'Other Investigators in Lab.Colony Counts.4.23.92-11.27.02.xlsx') 

ws = wb.get_sheet_by_name('Sheet1')
inv_colony = []

for i in range(3, 626):
    for col in ['D','E','F']:
        try:
            inv_colony.append(float(ws[col + str(i)].value))
        except:
            pass

In [329]:
# other lab colony data
# code from Aaron

DATA_DIR = '../data/'
wb = load_workbook(DATA_DIR + 'Outside Lab 3.Colony Counts.2.4.10-5.21.12.xlsx') 

ws = wb.get_sheet_by_name('Sheet1')
lab_colony = []

for i in range(3, 53):
    for col in ['B','C','D']:
        try:
            lab_colony.append(float(ws[col + str(i)].value))
        except:
            pass

# read data coulter

In [261]:
# RTS coulter data
# code from Aaron

DATA_DIR = '../data/'
wb = load_workbook(DATA_DIR + 'Bishayee Coulter Counts.10.20.97-7.16.01.xlsx') 

ws = wb.get_sheet_by_name('Sheet1')
rts_coulter = []

for i in range(3, 1732):
    for col in ['C','D','E']:
        try:
            rts_coulter.append(float(ws[col + str(i)].value))
        except:
            pass

In [262]:
# other investigator coulter data
# code from Aaron

wb = load_workbook(DATA_DIR + 'Other Investigators in Lab.Coulter Counts.4.15.92-5.21.05.xlsx') 

ws = wb.get_sheet_by_name('Sheet1')
inv_coulter = []

for i in range(3, 1010):
    for col in ['C','D','E']:
        try:
            inv_coulter.append(float(ws[col + str(i)].value))
        except:
            pass

In [263]:
# other lab coulter data
# code from Aaron

wb = load_workbook(DATA_DIR + 'Outside Lab 1.Coulter Counts.6.7.91-4.9.99.xlsx') 

ws = wb.get_sheet_by_name('Sheet1')
lab_coulter = []

for i in range(2, 111):
    for col in ['B','C','D']:
        try:
            lab_coulter.append(float(ws[col + str(i)].value))
        except:
            pass

# Permutation test - One versus all

## terminal digit analysis

Use Chi-squared distance / Total Variation Distance for the measure between distributions.

In [330]:
rts_colony_terminal = [np.mod(i, 10) 
                       for i in rts_colony]
lab_colony_terminal = [np.mod(i, 10) 
                       for i in lab_colony]
inv_colony_terminal = [np.mod(i, 10) 
                       for i in inv_colony]

In [310]:
def calc_distance(group1, group2):
    group1_obs = [group1.count(i)/len(group1) for i in range(10)]
    group2_obs = [group2.count(i)/len(group2) for i in range(10)]
    tvd_obs = 0.5*sum([abs(a_i - b_i) 
                       for a_i, b_i in zip(group1_obs, group2_obs)])
    chi_sq_obs = 0.5*sum([abs(a_i - b_i)**2/(a_i + b_i) 
                          for a_i, b_i in zip(group1_obs, group2_obs)])
    return tvd_obs, chi_sq_obs

In [313]:
def terminal_permutation(group1, group2, replication):
    tvd = []
    chi_sq = []
    for i in np.arange(replication):
        shuffled = random.sample(group1 + group2, 
                                 len(group1 + group2))
        tvd_shuffled, chi_sq_shuffled = calc_distance(shuffled[:len(group1)], 
                                                      shuffled[len(group1):])
        tvd.append(tvd_shuffled)
        chi_sq.append(chi_sq_shuffled)
    return tvd, chi_sq

permutation between RTS and other indivaduals

In [331]:
tvd_obs, chi_sq_obs = calc_distance(rts_colony_terminal, 
                                    inv_colony_terminal)
print("tvd_obs", tvd_obs)
print("chi_sq_obs", chi_sq_obs)

tvd_obs 0.09260466984966408
chi_sq_obs 0.011145925839597975


In [332]:
tvd, chi_sq = terminal_permutation(rts_colony_terminal, 
                                   inv_colony_terminal, 
                                   replication = 10000)

In [333]:
tvd_p_value = sum(np.array(tvd) > tvd_obs) / REPLICATION
chi_sq_p_value = sum(np.array(chi_sq) > chi_sq_obs) / REPLICATION
print("tvd p-value: ", tvd_p_value)
print("chi square p-value: ", chi_sq_p_value)

tvd p-value:  0.0
chi square p-value:  0.0


permutation between other lab and other indivaduals

In [334]:
tvd_obs, chi_sq_obs = calc_distance(inv_colony_terminal, 
                                    lab_colony_terminal)
print("tvd_obs", tvd_obs)
print("chi_sq_obs", chi_sq_obs)

tvd_obs 0.12830243547800801
chi_sq_obs 0.020362015799805375


In [335]:
tvd, chi_sq = terminal_permutation(inv_colony_terminal, 
                                   lab_colony_terminal, 
                                   replication = 10000)

In [336]:
tvd_p_value = sum(np.array(tvd) > tvd_obs) / REPLICATION
chi_sq_p_value = sum(np.array(chi_sq) > chi_sq_obs) / REPLICATION
print("tvd p-value: ", tvd_p_value)
print("chi square p-value: ", chi_sq_p_value)

tvd p-value:  0.1419
chi square p-value:  0.2685


## Equal digit analysis

In [275]:
def is_equal_digit(vec):
    vec = [i for i in vec if i >= 100]
    return [(count - count % 10)/10 % 10  == count % 10 for count in vec]

In [280]:
def binomial_stat(group1, group2):
    prop1 = sum(group1) / len(group1)
    prop2 = sum(group2) / len(group2)
    return abs(prop1 - prop2) / np.sqrt(prop1*(1-prop1) / len(group1) + prop2*(1-prop2) / len(group2))

In [293]:
def binomial_permutation(group1, group2, replication):
    stats = []
    for i in np.arange(replication):
        shuffled = random.sample(group1 + group2, 
                                 len(group1 + group2))
        stat_shuffled = binomial_stat(shuffled[:len(group1)], shuffled[len(group1):])
        stats.append(stat_shuffled)
    return stats

In [276]:
rts_coulter_equal = is_equal_digit(rts_coulter)
inv_coulter_equal = is_equal_digit(inv_coulter)
lab_coulter_equal = is_equal_digit(lab_coulter)

In [296]:
# permutation between RTS and other indivaduals
equal_digit_obs = binomial_stat(rts_coulter_equal, 
                                inv_coulter_equal)
equal_digit_permute = binomial_permutation(rts_coulter_equal, 
                                           inv_coulter_equal, 
                                           replication = 10000)

In [298]:
# pvalue of permutation test between RTS and other indivaduals
rts_inv_p_value = sum(np.array(equal_digit_permute) > equal_digit_obs) / replication
rts_inv_p_value

0.00040000000000000002

In [301]:
# permutation between other indivaduals and other labs
equal_digit_obs = binomial_stat(inv_coulter_equal, 
                                lab_coulter_equal)
equal_digit_permute = binomial_permutation(inv_coulter_equal, 
                                           lab_coulter_equal, 
                                           replication = 10000)

In [302]:
# pvalue of permutation test between other labs and other indivaduals
inv_lab_p_value = sum(np.array(equal_digit_permute) > equal_digit_obs) / replication
inv_lab_p_value

0.76939999999999997